In [1]:
from pandas.io import data, wb
import datetime
import os.path
import math
import pandas as pd
import numpy

def update_ohlc(symbol):
    
    today = datetime.datetime.today()
    
    #update ohlc data
    if (not os.path.isfile('data/'+symbol+'-ohlc.csv')):
        df_train = data.DataReader(symbol, 'yahoo', datetime.datetime(1900, 2, 1), today)

    else:
        df_train = pd.read_csv('data/'+symbol+'-ohlc.csv', index_col='Date', parse_dates=True)
        start = df_train.index[-1] + datetime.timedelta(days=1)
        print(start,today)

        if(start <= today):
            new_data = data.DataReader(symbol, 'yahoo', start, today )
            print(new_data)
            df_train = df_train.append(new_data)

    df_train.to_csv('data/'+symbol+'-ohlc.csv')
    
    #update dividends info
    filename_dividends = 'data/'+symbol+'-dividends.csv'

    
    url_temp = 'http://ichart.yahoo.com/table.csv?s={s}&c={c}&a={a}&b={b}&f={f}&d={d}&e={e}&g=v&ignore=.csv'
    #if (not os.path.isfile(filename_dividends)): #create new dividend file
    url = url_temp.format(s=symbol, a='1', b='1', c='1980',d=today.month, e=today.day, f=today.year)
    print(url)
    df_dividends = pd.read_csv(url, index_col=0, parse_dates=[0])
    if len(df_dividends) > 0:        
        #sort to ascending order, match that of ohlc
        df_dividends.sort_index(inplace=True)
        df_dividends.to_csv(filename_dividends) 
        
    '''
    else: #append new dividend info
        df_dividends = pd.read_csv(filename_dividends, index_col='Date', parse_dates=True)
        start = df_dividends.index[-1] + datetime.timedelta(days=1)
        print(start)
        if(start == today):
            url = url_temp.format(s=symbol, a=start.month, b=start.day, c=start.year,d=today.month, e=today.day, f=today.year)
            print(url)
            new_data = pd.read_csv(url, index_col=0, parse_dates=[0])
            if len(new_data) > 0:
                #sort to ascending order, match that of ohlc
                new_data.sort_index(inplace=True)
                print(new_data)
                df_dividends = df_dividends.append(new_data)
    
     '''
        
symbols = ['SPY', 'QQQ','IWM', 'EEM', 'AAPL', 'GOOG', 'XOM', 'FB', 'JNJ','GE',  'AMZN','WFC', 'T', 'PG', 'WMT', 'JPM', 'VZ', 'KO',
           'PFE',  'V', 'HD', 'CVX', 'DIS', 'INTC', 'IBM', 'GS','DVA', 'MO', 'GILD', 'TSLA', 'GLD']
#symbols = ['GLD']
for symbol in symbols:
    update_ohlc(symbol)
print("update completed.")

C:\Anaconda3\lib\site-packages\pandas\io\data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)
C:\Anaconda3\lib\site-packages\pandas\io\wb.py:21: FutureWarning: 
The pandas.io.wb module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


2016-04-28 00:00:00 2016-04-28 22:47:16.507470
                  Open        High         Low       Close    Volume  \
Date                                                                   
2016-04-28  208.460007  209.759995  206.960007  207.449997  92734800   

             Adj Close  
Date                    
2016-04-28  207.449997  
http://ichart.yahoo.com/table.csv?s=SPY&c=1980&a=1&b=1&f=2016&d=4&e=28&g=v&ignore=.csv
2016-04-28 00:00:00 2016-04-28 22:47:17.297913
                  Open        High     Low       Close    Volume   Adj Close
Date                                                                        
2016-04-28  107.699997  108.330002  106.07  106.279999  29586500  106.279999
http://ichart.yahoo.com/table.csv?s=QQQ&c=1980&a=1&b=1&f=2016&d=4&e=28&g=v&ignore=.csv
2016-04-28 00:00:00 2016-04-28 22:47:17.684336
                  Open   High         Low  Close    Volume  Adj Close
Date                                                                 
2016-04-28  114.269997